# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откроем файл с данными и изучим общую информацию

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
print(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   с

**Вывод**

В столбцах days_employed и total_income есть пропуски. Надо проверить эти записи и либо заполнить либо удалить. В столбце days_employed отрицательные значения, хотя судя по названию должны быть целые числа. Столбец education видно что категориальный но в разных регистрах. Надо привести к нижнему и проверить будет ли совпадать количество записей и education_id.  столбец purpose тоже надо обработать и привести к категориям.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
print(data[data['days_employed'].isna()].head(10))

    children  days_employed  dob_years education  education_id  \
12         0            NaN         65   среднее             1   
26         0            NaN         41   среднее             1   
29         0            NaN         63   среднее             1   
41         0            NaN         50   среднее             1   
55         0            NaN         54   среднее             1   
65         0            NaN         21   среднее             1   
67         0            NaN         52    высшее             0   
72         1            NaN         32    высшее             0   
82         2            NaN         50    высшее             0   
83         0            NaN         52   среднее             1   

            family_status  family_status_id gender  income_type  debt  \
12       гражданский брак                 1      M    пенсионер     0   
26        женат / замужем                 0      M  госслужащий     0   
29  Не женат / не замужем                 4      F    

Если удалить строки из столбца days_employed, то это может повлиять на далбнейшие расчеты. Подчситаем сколько таких пропусков.

In [3]:
print(len(data[data['days_employed'].isna()]))

2174


Из 21525 записей 2174 (10%) с прпусками удалять точно не надо. Заменим их на среднее значение, так как это может повлиять на дальнейшие расчеты.

In [4]:
days_employed_mean = data['days_employed'].mean()
data['days_employed'] = data['days_employed'].fillna(days_employed_mean)
print(data[data['days_employed'].isna()].head(10))

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


теперь проверить пропущенные данные столбца total_income

In [5]:
print(data[data['total_income'].isna()].head(100))

     children  days_employed  dob_years education  education_id  \
12          0   63046.497661         65   среднее             1   
26          0   63046.497661         41   среднее             1   
29          0   63046.497661         63   среднее             1   
41          0   63046.497661         50   среднее             1   
55          0   63046.497661         54   среднее             1   
..        ...            ...        ...       ...           ...   
959         0   63046.497661         25   среднее             1   
973         0   63046.497661         49   среднее             1   
974         0   63046.497661         60   среднее             1   
978         3   63046.497661         28   среднее             1   
980         0   63046.497661         53    высшее             0   

             family_status  family_status_id gender  income_type  debt  \
12        гражданский брак                 1      M    пенсионер     0   
26         женат / замужем                 0   

Очевидно что это те же клиенты у которых отсутствовали записи в колонке days_employed. Так же заменим пропуски на среднее значение доходов.

In [6]:
total_income_mean = data['total_income'].mean()
data['total_income'] = data['total_income'].fillna(total_income_mean)
print(data[data['total_income'].isna()].count())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


Пропусков больше нет.

**Вывод**

Возможно пропуски образованы по причине определенной категории клиентов, например те кто уже брал кредит в в этом банке или доверенные клиенты банка партнера.

### Замена типа данных

Теперь вопрос к данным в столбце days_employed. Врят ли может быть отрицательным количество рабочих дней. Скорее всего отсчет идет от какой то конкретной даты. А то что данные имеют формат вещественного числа наводит на мысль об учете времени, что нам совсем не актуально. Приведем эти данные к целочисленному значению.

In [7]:
data['days_employed'] = data['days_employed'].astype('int')
print(data['days_employed'].head(10))

0     -8437
1     -4024
2     -5623
3     -4124
4    340266
5      -926
6     -2879
7      -152
8     -6929
9     -2188
Name: days_employed, dtype: int64


найдем максимальное и минимальон значение.

In [8]:
print('Максимальное значение:', data['days_employed'].max())
print('Минимальное значение:', data['days_employed'].min())

Максимальное значение: 401755
Минимальное значение: -18388


Теперь приведем данные к положительным значениям. Найдем минимальое и увеличим значение данных на эту величину. теперь похоже что подсчет велся в часах.

In [9]:
days_employed_min = abs(data['days_employed'].min())
data['days_employed'] = (data['days_employed'] + days_employed_min) / 24
data['days_employed'] = data['days_employed'].astype('int')
print(data['days_employed'].head(10))

0      414
1      598
2      531
3      594
4    14943
5      727
6      646
7      759
8      477
9      675
Name: days_employed, dtype: int64


Так же приведем к целому типу столбец total_income методом astype так как с целыми числами работать проще и занимает меньше памяти

In [10]:
data['total_income'] = data['total_income'].astype('int')
print(data['total_income'].head())

0    253875
1    112080
2    145885
3    267628
4    158616
Name: total_income, dtype: int64


**Вывод**

Данные приведены к форме количества отработанных дней. Стаж клиента тоже может влиять на вероятность выплаты кредита.

### Обработка дубликатов

Проверим количество дубликатов

In [11]:
print(data.duplicated().value_counts())

False    21471
True        54
dtype: int64


Проверим данные в столбце education

In [12]:
print(data['education'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


Видны повторения из за разных регистров, приведем все данные в нижний регистр

In [13]:
data['education'] = data['education'].str.lower()
print(data['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


**Вывод**

каак выясняется больше половины клиентов со средним образованием

In [14]:
for column in data.columns:
    print(data[column].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
3393     2174
757       203
756       173
758       172
755       160
         ... 
16863       1
14800       1
16847       1
14784       1
16376       1
Name: days_employed, Length: 2631, dtype: int64
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64
среднее                15233
высшее                  52

### Лемматизация

Теперь надо данные в столбце purpose разбить на категории. Для этого импортируем библиотеку pymystem3 и Counter из collections

In [15]:
from pymystem3 import Mystem
m = Mystem()

посмотрим какие есть леммы в столбце purpose

In [16]:
from collections import Counter
Counter(data['purpose'].apply(m.lemmatize).sum()) 

Counter({'покупка': 5912,
         ' ': 33677,
         'жилье': 4473,
         '\n': 21525,
         'приобретение': 462,
         'автомобиль': 4315,
         'дополнительный': 909,
         'образование': 4022,
         'сыграть': 774,
         'свадьба': 2348,
         'операция': 2610,
         'с': 2924,
         'на': 2233,
         'проведение': 777,
         'для': 1294,
         'семья': 641,
         'недвижимость': 6367,
         'коммерческий': 1315,
         'жилой': 1233,
         'строительство': 1881,
         'собственный': 635,
         'подержать': 479,
         'свой': 2235,
         'со': 630,
         'заниматься': 908,
         'сделка': 944,
         'подержанный': 489,
         'получение': 1316,
         'высокий': 1375,
         'профильный': 436,
         'сдача': 653,
         'ремонт': 612})

**Вывод**

После лемматизации видно 4 катерории получения кредита автомобиль, недвижимость, свадьба и образование.

### Категоризация данных

Создадим новый столбец с категориями

In [17]:
def category_purpose(purpose):
    lem_purpose = m.lemmatize(purpose)
    if 'автомобиль' in lem_purpose:
        return 'автомобиль'
    if 'жилье' in lem_purpose or 'недвижимость' in lem_purpose:
        return 'недвижимость'
    if 'свадьба' in lem_purpose:
        return 'свадьба'
    if 'образование' in lem_purpose:
        return 'образование'
data['purpose_category'] = data['purpose'].apply(category_purpose)
print(data.groupby('purpose_category')['purpose'].count())

purpose_category
автомобиль       4315
недвижимость    10840
образование      4022
свадьба          2348
Name: purpose, dtype: int64


Разобьем клиентов на 2 категории, у кого есть дети и у кого нет

In [18]:
print(data['children'].unique())

[ 1  0  3  2 -1  4 20  5]


не может быть отрицательного значения количества детей, возможно это ошибка ввода.

In [19]:
def category_children(children):
    if children == 0:
        return 'нет детей'
    else: return 'есть дети'
data['children_category'] = data['children'].apply(category_children)
print(data.groupby('children_category')['children'].count())

children_category
есть дети     7376
нет детей    14149
Name: children, dtype: int64


разобъем уровень дохода на категории и для начала определим минмальный и максимальный доход

In [20]:
print('Минимальный доход', data['total_income'].min())
print('Максимальный доход', data['total_income'].max())
print('Средний доход', data['total_income'].median())

Минимальный доход 20667
Максимальный доход 2265604
Средний доход 156400.0


разделим на 3 группы, до 100 000 руб низкий доход, от 100 000 до 1 000 000 средний доход и свыше 1 000 000 высокий. 

In [21]:
def category_total_income(total_income):
    if total_income <= 100000:
        return 'низкий'
    if 100000 < total_income <= 1000000:
        return 'средний'
    else: return 'высокий'
data['total_income_category'] = data['total_income'].apply(category_total_income)
print(data['total_income_category'].value_counts())

средний    17037
низкий      4463
высокий       25
Name: total_income_category, dtype: int64


**Вывод**

Узнав категории получения кредита видно что самый популярный это недвижимость и кредиты берут в основном со средним уровнем дохода

## Шаг 3. Ответ на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [22]:
print(data['debt'].value_counts()) #посмотрим сколько всего должников

0    19784
1     1741
Name: debt, dtype: int64


In [23]:
def debt_children (row):
    children = row['children_category']
    debt = row['debt']
    if debt == 1:
        if children == 'нет детей':
            return 'зависит'
    return 'не зависит'
data['debt_children_category'] = data.apply(debt_children, axis=1)
print(data['children_category'].value_counts())
print(data['debt_children_category'].value_counts())

нет детей    14149
есть дети     7376
Name: children_category, dtype: int64
не зависит    20462
зависит        1063
Name: debt_children_category, dtype: int64


**Вывод**

1063 совпадения говорит о том что 61% должников не имеют детей

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [24]:
print(data['family_status'].value_counts())

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


In [25]:
def debt_family_status (row):
    family_status = row['family_status_id']
    debt = row['debt']
    if debt == 1:
        if family_status >= 2:
            return 'зависит'
    return 'не зависит'
data['debt_family_status_category'] = data.apply(debt_family_status, axis=1)
print(data['debt_family_status_category'].value_counts())

не зависит    21103
зависит         422
Name: debt_family_status_category, dtype: int64


**Вывод**

из 1741 клиента 422 не имеют семьи и это составляет 25%

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [26]:
def debt_total_income (row):
    total_income = row['total_income_category']
    debt = row['debt']
    if debt == 1:
        if total_income == 'низкий':
            return 'низкий доход'
        if total_income == 'средний':
            return 'средний доход'
        if total_income == 'высокий':
            return 'высокий доход'
    return 'не зависит'
data['debt_total_income_category'] = data.apply(debt_total_income, axis=1)
print(data['debt_total_income_category'].value_counts())

не зависит       19784
средний доход     1385
низкий доход       354
высокий доход        2
Name: debt_total_income_category, dtype: int64


**Вывод**

80% несвоевременного возврата у клиентов со средним доходом

- Как разные цели кредита влияют на его возврат в срок?

In [27]:
def debt_purpose (row):
    purpose_category = row['purpose_category']
    debt = row['debt']
    if debt == 1:
        if purpose_category == 'автомобиль':
            return 'на автомобиль'
        if purpose_category == 'недвижимость':
            return 'на недвижимость'
        if purpose_category == 'образование':
            return 'на образование'
        if purpose_category == 'свадьба':
            return 'на свадьбу'
    return 'не зависит'
data['debt_purpose_category'] = data.apply(debt_purpose, axis=1)
print(data['debt_purpose_category'].value_counts())

не зависит         19784
на недвижимость      782
на автомобиль        403
на образование       370
на свадьбу           186
Name: debt_purpose_category, dtype: int64


**Вывод**

Больше несвоевременных возвратов за кредиты на недвижимость, но у автокредитов больше процент должников

In [28]:
purpose_table = data.pivot_table(index='purpose_category', columns='debt', values='days_employed', aggfunc='count')
purpose_table['percent'] = purpose_table[1] / (purpose_table[0] + purpose_table[1]) * 100 
print(purpose_table)

debt                  0    1   percent
purpose_category                      
автомобиль         3912  403  9.339513
недвижимость      10058  782  7.214022
образование        3652  370  9.199403
свадьба            2162  186  7.921635


## Шаг 4. Общий вывод

Наличие детей и семейное положение влияют на вероятность несвоевременных возвратов кредита, особенное если у клиента средний уровень дохода. А вот цель кредита сильно не влияет на процент невозврата.